In [2]:
# !pip install openquake.engine   

In [3]:
# Basic python modules
import numpy as np
import matplotlib.pyplot as plt
import os

# OQ modules
from openquake.hazardlib.gsim import get_available_gsims
from openquake.hazardlib.gsim.base import RuptureContext, SitesContext, DistancesContext
from openquake.hazardlib.imt import SA
from openquake.hazardlib import const

In [4]:
# Ground Motion Prediction Equation

gmpe_list = get_available_gsims()
gmpe_list

{'AbrahamsonEtAl2014': openquake.hazardlib.gsim.abrahamson_2014.AbrahamsonEtAl2014,
 'AbrahamsonEtAl2014NSHMPLower': openquake.hazardlib.gsim.nshmp_2014.NSHMP2014,
 'AbrahamsonEtAl2014NSHMPMean': openquake.hazardlib.gsim.nshmp_2014.NSHMP2014,
 'AbrahamsonEtAl2014NSHMPUpper': openquake.hazardlib.gsim.nshmp_2014.NSHMP2014,
 'AbrahamsonEtAl2014RegCHN': openquake.hazardlib.gsim.abrahamson_2014.AbrahamsonEtAl2014,
 'AbrahamsonEtAl2014RegJPN': openquake.hazardlib.gsim.abrahamson_2014.AbrahamsonEtAl2014,
 'AbrahamsonEtAl2014RegTWN': openquake.hazardlib.gsim.abrahamson_2014.AbrahamsonEtAl2014,
 'AbrahamsonEtAl2015SInter': openquake.hazardlib.gsim.abrahamson_2015.AbrahamsonEtAl2015SInter,
 'AbrahamsonEtAl2015SInterHigh': openquake.hazardlib.gsim.abrahamson_2015.AbrahamsonEtAl2015SInterHigh,
 'AbrahamsonEtAl2015SInterLow': openquake.hazardlib.gsim.abrahamson_2015.AbrahamsonEtAl2015SInterLow,
 'AbrahamsonEtAl2015SInter_scaled': openquake.hazardlib.gsim.abrahamson_2015.AbrahamsonEtAl2015SInter_sca

In [5]:
gmpe_list = get_available_gsims()
GMPE_PARAMETERS = gmpe_list["AbrahamsonGulerce2020SSlab"]()
print(GMPE_PARAMETERS.REQUIRES_RUPTURE_PARAMETERS)
print(GMPE_PARAMETERS.REQUIRES_SITES_PARAMETERS)
print(GMPE_PARAMETERS.REQUIRES_DISTANCES)

frozenset({'mag', 'ztor'})
frozenset({'vs30'})
frozenset({'rrup'})


In [6]:
rctx = RuptureContext()
rctx.mag = np.array([7.9])
rctx.hypo_depth = np.array([90])
rctx.ztor = np.array([80])

sctx = SitesContext()
sctx.vs30 = np.array([760.])
sctx.backarc = True
sctx.sids = np.array([0])

dctx = DistancesContext()
dctx.rrup = np.array([100])
dctx.rhypo = np.array([100])

In [8]:
periods = np.logspace(-2, 1, 30)
# 
imts = [SA(period) for period in periods]

stddev_types = [const.StdDev.TOTAL]

In [9]:
means = np.zeros(len(imts))
means_plus_1sd = np.zeros(len(imts))
means_minus_1sd = np.zeros(len(imts))
stddevs = np.zeros(len(imts))

gmpes = [gmpe_list["MontalvaEtAl2017SSlab"](),
         gmpe_list["ParkerEtAl2020SSlab"]()]

In [10]:
def calculate_ground_motions(gmpes, imts, sctx, rctx, dctx, stddev_types):
    """
    Calculates the expected ground motion and uncertainty, organised by GMPE
    and intensity measure type (i.e. PGA, SA etc.), for a given rupture-site configuration
    """
    results = {}
    nper = len(imts)
    for gmpe in gmpes:
        print("Running GMPE %s" % str(gmpe))
        results[str(gmpe)] = {"mean": np.zeros(nper),
                              "stddevs": np.zeros(nper),
                              "mean_plus_1sd": np.zeros(nper),
                              "mean_minus_1sd": np.zeros(nper)}
        for i, imt in enumerate(imts):
            try:
                mean, [stddev] = gmpe.get_mean_and_stddevs(
                    sctx, rctx, dctx, imt, stddev_types)
                results[gmpe]["mean"][i] = np.exp(mean)
                results[gmpe]["stddevs"][i] = stddev
                results[gmpe]["mean_plus_1sd"][i] = np.exp(mean + stddev)
                results[gmpe]["mean_minus_1sd"][i] = np.exp(mean - stddev)
            except KeyError:
                results[gmpe]["mean"][i] = np.nan
                results[gmpe]["stddevs"][i] = np.nan
                results[gmpe]["mean_plus_1sd"][i] = np.nan
                results[gmpe]["mean_minus_1sd"][i] = np.nan
    return results


In [11]:
results = calculate_ground_motions(gmpes, imts, sctx, rctx, dctx, stddev_types)

Running GMPE [MontalvaEtAl2017SSlab]
Running GMPE [ParkerEtAl2020SSlab]


/var/folders/xt/h6m5q8xs5qs7f905tg1pwtz80000gn/T/ipykernel_84009/669690744.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  results[gmpe]["mean"][i] = np.exp(mean)
/var/folders/xt/h6m5q8xs5qs7f905tg1pwtz80000gn/T/ipykernel_84009/669690744.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  results[gmpe]["stddevs"][i] = stddev
/var/folders/xt/h6m5q8xs5qs7f905tg1pwtz80000gn/T/ipykernel_84009/669690744.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  results[

In [12]:
for key in results.keys():
    print(str(key))
    print(results[key])

[MontalvaEtAl2017SSlab]
{'mean': array([0.18483621, 0.18719261, 0.18957906, 0.1942584 , 0.22357497,
       0.25632637, 0.29282488, 0.33786143, 0.40553408, 0.45952804,
       0.47987756, 0.46625134, 0.43662759, 0.37465787, 0.27563998,
       0.20114655, 0.15292629, 0.11902662, 0.09210968, 0.07048947,
       0.05293174, 0.03936394, 0.02887821, 0.02281503, 0.01717217,
       0.01243027, 0.00869041, 0.00611052, 0.00505651, 0.00413013]), 'stddevs': array([0.83844918, 0.84110491, 0.84376063, 0.84706472, 0.85692103,
       0.86677733, 0.87663364, 0.88760494, 0.90204425, 0.90901477,
       0.9079249 , 0.90338391, 0.88436829, 0.86310782, 0.84420147,
       0.83276501, 0.81203524, 0.80134624, 0.81017091, 0.80974758,
       0.8023014 , 0.79204019, 0.76837164, 0.75729074, 0.73362281,
       0.69128478, 0.67692496, 0.64379338, 0.62427024, 0.6024269 ]), 'mean_plus_1sd': array([0.42748502, 0.43408615, 0.4407892 , 0.45316393, 0.52671933,
       0.60985966, 0.70359886, 0.82076825, 0.999494  , 1.1404916